In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
from typing_extensions import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [3]:
import os, requests
resp = requests.get(
    "https://api.groq.com/openai/v1/models",
    headers={"Authorization": f"Bearer {os.getenv('GROQ_API_KEY')}", "Content-Type": "application/json"},
    timeout=30
)
print(resp.json())

{'object': 'list', 'data': [{'id': 'qwen/qwen3-32b', 'object': 'model', 'created': 1748396646, 'owned_by': 'Alibaba Cloud', 'active': True, 'context_window': 131072, 'public_apps': None, 'max_completion_tokens': 40960}, {'id': 'meta-llama/llama-prompt-guard-2-22m', 'object': 'model', 'created': 1748632101, 'owned_by': 'Meta', 'active': True, 'context_window': 512, 'public_apps': None, 'max_completion_tokens': 512}, {'id': 'openai/gpt-oss-120b', 'object': 'model', 'created': 1754408224, 'owned_by': 'OpenAI', 'active': True, 'context_window': 131072, 'public_apps': None, 'max_completion_tokens': 65536}, {'id': 'canopylabs/orpheus-arabic-saudi', 'object': 'model', 'created': 1765926439, 'owned_by': 'Canopy Labs', 'active': True, 'context_window': 4000, 'public_apps': None, 'max_completion_tokens': 50000}, {'id': 'groq/compound', 'object': 'model', 'created': 1756949530, 'owned_by': 'Groq', 'active': True, 'context_window': 131072, 'public_apps': None, 'max_completion_tokens': 8192}, {'id'

In [4]:
llm = init_chat_model("llama-3.3-70b-versatile",model_provider="groq")

In [5]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)

builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [8]:
message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages": [message]})

response["messages"]

[HumanMessage(content='What is the latest price of MSFT stock?', additional_kwargs={}, response_metadata={}, id='601930f6-5ff6-4637-9ccd-c1d6ff57e32f'),
 AIMessage(content="I'm not able to provide real-time stock prices. However, I can suggest some options for you to find the current price of MSFT stock:\n\n1. Check a financial website: Websites like Yahoo Finance, Google Finance, or Bloomberg provide up-to-date stock prices. You can visit their website and search for MSFT to find the current price.\n2. Use a stock market app: Many stock market apps, such as Robinhood, Fidelity, or E\\*TRADE, provide real-time stock prices. You can download one of these apps and search for MSFT to find the current price.\n3. Look up a financial news website: Websites like CNBC, Forbes, or The Wall Street Journal often provide real-time stock prices and market updates.\n\nPlease note that stock prices can fluctuate rapidly, so the price may change quickly. It's always a good idea to verify the price wit

In [7]:
state = None
while True:
    in_message = input("User: ")
    if in_message.lower() in ["exit", "quit"]:
        break
    if state is None:
        state = {"messages": [{"role": "user", "content": in_message}]}
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Chatbot:", state["messages"][-1].content)

Chatbot: It looks like you may have accidentally sent a blank message. Please feel free to start again, and I'll do my best to help with any questions or topics you'd like to discuss. What's on your mind today?
Chatbot: The New York Stock Exchange (NYSE) is home to over 2,400 listed companies, and the top stocks can vary depending on the criteria used to evaluate them. Here are some of the top stocks in the NYSE, categorized by market capitalization, performance, and sector:

**Top 10 Stocks by Market Capitalization:**

1. **Apple Inc. (AAPL)** - Market cap: over $2.3 trillion
2. **Microsoft Corporation (MSFT)** - Market cap: over $2.2 trillion
3. **Alphabet Inc. (GOOGL)** - Market cap: over $1.3 trillion
4. **Amazon.com, Inc. (AMZN)** - Market cap: over $1.2 trillion
5. **Facebook, Inc. (FB)** - Market cap: over $850 billion
6. **Berkshire Hathaway Inc. (BRK.A)** - Market cap: over $530 billion
7. **Johnson & Johnson (JNJ)** - Market cap: over $430 billion
8. **Procter & Gamble Compan